In [118]:
import pandas as pd
import numpy as np


#dialogs_1_data = pd.read_csv("data/DATA/dialogs_1.csv")
dialogs_2_data = pd.read_csv("data/DATA/dialogs_2.csv", parse_dates=["datetime"])
#email_reports = pd.read_csv("data/DATA/email_reports.csv")


In [119]:
#dialogs_1_data.head()

In [120]:
#dialogs_2_data.head()

In [121]:
#email_reports.head()

In [122]:
dialogs_2_data.dropna(how="all", subset=["client_id", "user_id"], inplace=True)
clear_data = dialogs_2_data.reset_index()

In [123]:
clear_data["index"] = clear_data.index.astype(float)
clear_data.rename(columns={'index': 'message_id'}, inplace=True)
#clear_data.user_id = clear_data.user_id.astype(int, errors = 'ignore')

In [124]:
clear_data["author_id"] = clear_data[["client_id", "user_id"]].sum(axis=1)

In [207]:
clear_data = clear_data[0:10000]

In [208]:
clear_data.loc[clear_data["client_id"] > 0, "author_type"] = "Client"
clear_data.loc[clear_data["client_id"].isnull(),"author_type"] = "Operator"

In [209]:
clear_data.dtypes

message_id             float64
appeal_id                int64
datetime        datetime64[ns]
client_id              float64
user_id                float64
msg_type                 int64
channel_id               int64
channel_name            object
channel_type            object
answer_time            float64
text                    object
author_id              float64
author_type             object
datetime_end    datetime64[ns]
dtype: object

In [210]:
clear_data.head()

,message_id,appeal_id,datetime,client_id,user_id,msg_type,channel_id,channel_name,channel_type,answer_time,text,author_id,author_type,datetime_end
0,0.0,323874,2088-08-08 13:44:02,NaN,19670.0,0,73,Webdealer,Веб-чат,12.0,Привет.)),19670.0,Operator,2088-08-08 13:44:03
1,1.0,323870,2088-08-08 13:44:01,NaN,19671.0,0,73,Webdealer,Веб-чат,NaN,Все подписки отключены. Как это делать самосто...,19671.0,Operator,2088-08-08 13:44:02
2,2.0,323861,2088-08-08 13:44:01,NaN,19418.0,0,133,Нижний Новгород B2C,Веб-чат,NaN,Вы забыли пароль и хотите войти в кабинет?,19418.0,Operator,2088-08-08 13:44:02
3,3.0,323850,2088-08-08 13:44:01,NaN,19578.0,0,2,Viber,Viber,54.0,Можем отправить заявку на подключение новых но...,19578.0,Operator,2088-08-08 13:44:02
4,4.0,323867,2088-08-08 13:43:58,569008.0,NaN,0,177,Омск B2C,Веб-чат,NaN,Тогда проще перейти с номером в мегафон Включа...,569008.0,Client,2088-08-08 13:43:59


Prepare data

In [211]:
import featuretools as ft

In [212]:
# Fix of https://stackoverflow.com/questions/51191665/excluding-the-current-row-from-feature-engineering-in-python-featuretools
# Fix of https://github.com/Featuretools/featuretools/issues/184
clear_data['datetime_end'] = clear_data['datetime'] + pd.Timedelta(seconds=1)

In [277]:
es = ft.EntitySet(id='tele2')

In [295]:
es = es.entity_from_dataframe('message'
                              , clear_data
                              , index="message_id"
                              , time_index="datetime"
#                               , secondary_time_index={'datetime_end': ["client_id"
#                                     ,"user_id"
#                                     ,"msg_type"
#                                     ,"channel_id"
#                                     ,"channel_name"
#                                     ,"channel_type"
#                                     ,"answer_time"
#                                     ,"text"
#                                      ,"author_id"
#                                      ,"author_type"]}
#                              , last_time_index="datetime_end"
                             )
es

Entityset: tele2
  Entities:
    message [Rows: 10000, Columns: 14]
    dialogs [Rows: 791, Columns: 3]
  Relationships:
    message.appeal_id -> dialogs.appeal_id

In [297]:
es.normalize_entity(base_entity_id="message",
    new_entity_id="dialogs",
    index="appeal_id"
#     , make_time_index="datetime"
#     , time_index_reduce = 'first'
    
#     , additional_variables=["client_id"
#                             ,"user_id"
#                             ,"msg_type"
#                             ,"channel_id"
#                             ,"channel_name"
#                             ,"channel_type"
#                             ,"answer_time"
#                             ,"text"
#                              ,"author_id"
#                              ,"author_type"
#                            ]
#     , additional_variables=["datetime","datetime_end"]
#     , make_secondary_time_index = {"datetime_end": ["appeal_id"]}
)
es["dialogs"].df.head()

2018-08-19 08:50:48,614 featuretools.entityset - WARNING    Not adding duplicate relationship: <Relationship: message.appeal_id -> dialogs.appeal_id>


,appeal_id,first_message_time
323057,323057,2088-08-08 08:47:46
322678,322678,2088-08-08 08:47:49
322682,322682,2088-08-08 08:47:49
323179,323179,2088-08-08 08:47:50
323181,323181,2088-08-08 08:47:50


In [302]:
es.normalize_entity(base_entity_id="message",
    new_entity_id="author",
    index="author_id"
                        , additional_variables=["client_id"
#                             ,"user_id"
#                             ,"msg_type"
#                             ,"channel_id"
#                             ,"channel_name"
#                             ,"channel_type"
#                             ,"answer_time"
#                             ,"text"
#                              ,"author_id"
                             ,"author_type"
                           ]
)

2018-08-19 08:51:46,522 featuretools.entityset - WARNING    Not adding duplicate relationship: <Relationship: message.author_id -> author.author_id>


Entityset: tele2
  Entities:
    message [Rows: 10000, Columns: 12]
    dialogs [Rows: 791, Columns: 2]
    author [Rows: 725, Columns: 4]
  Relationships:
    message.appeal_id -> dialogs.appeal_id
    message.author_id -> author.author_id

In [303]:
len(es["author"].df.query("author_type == 'Operator'"))

56

In [304]:
# new_relationship = ft.Relationship(es["message"]["message_id"],
# es["author"]["author_id"])

# es = es.add_relationship(new_relationship)
# es

In [305]:
es.add_last_time_indexes()

In [306]:
es["author"].df.head()

,author_id,client_id,author_type,first_message_time
568539.0,568539.0,568539.0,Client,2088-08-08 08:47:46
473053.0,473053.0,473053.0,Client,2088-08-08 08:47:49
568332.0,568332.0,568332.0,Client,2088-08-08 08:47:49
568604.0,568604.0,568604.0,Client,2088-08-08 08:47:50
568606.0,568606.0,568606.0,Client,2088-08-08 08:47:50


In [309]:
feature_matrix, feature_defs = ft.dfs(entityset=es,
target_entity="author",
# cutoff_time=cutoff_times,
cutoff_time_in_index=True,
training_window="1 hour"
)

In [310]:
feature_matrix.head()

,,client_id,author_type,SUM(message.user_id),SUM(message.msg_type),SUM(message.channel_id),SUM(message.answer_time),STD(message.user_id),STD(message.msg_type),STD(message.channel_id),STD(message.answer_time),...,NUM_UNIQUE(message.WEEKDAY(datetime)),NUM_UNIQUE(message.WEEKDAY(datetime_end)),MODE(message.DAY(datetime)),MODE(message.DAY(datetime_end)),MODE(message.YEAR(datetime)),MODE(message.YEAR(datetime_end)),MODE(message.MONTH(datetime)),MODE(message.MONTH(datetime_end)),MODE(message.WEEKDAY(datetime)),MODE(message.WEEKDAY(datetime_end))
author_id,time,,,,,,,,,,,,,,,,,,,,,
19389.0,2018-08-19 08:52:37.506389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19391.0,2018-08-19 08:52:37.506389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19395.0,2018-08-19 08:52:37.506389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19396.0,2018-08-19 08:52:37.506389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19403.0,2018-08-19 08:52:37.506389,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
